In [ ]:
# ###################################
#     ----    S T A R T ----
# ###################################

# Libraries to be installed every run

!pip install moviepy;
!pip install pytube ;
!apt install imagemagick;


# ###################################
#       -- I M P O R T A N T --
# ###################################
# imagemagick have a piece of line that needs deleting so this would read and delete the line ( rewrite file )

policypath = '/etc/ImageMagick-6/policy.xml'
try:
  with open(policypath,'r') as f:
    lines = f.readlines()
  # del policypath
except:
  print('File not found...')

# policypath = '/etc/ImageMagick-6/policy.xml'
with open(policypath,'w') as rwrite:
  for i in lines:
    if i.strip() == '<policy domain="path" rights="none" pattern="@*"/>':
      print('deleted')
    else:
      rwrite.write(i)

In [ ]:
import sys
import os
from pytube import YouTube # downloading videos
from moviepy.editor import * # for editing video
from moviepy import *
import subprocess # splitting videos
import re

In [ ]:
def download(Link):
    cwdpath = os.getcwd()
    print(f' Your location : {cwdpath}')

    # DOWNLOAD YOUTUBE VIDEOS
    print(' Let\'s download a Video ')
    yt = YouTube(Link)
    title = yt.title
    print(f' Video Name      : {title}')
    desc = yt.description
    lenght = yt.length
    author = yt.author
    print(f' Length of video : {lenght} seconds')

    video = yt.streams.get_highest_resolution()
    vid_loc = os.path.join(cwdpath,'contents')
    path = video.download(vid_loc)

    document = {
        'Title'          : title.strip(),
        'Description'    : desc,
        'Video_Path'     : path,
        'Current_Folder' : cwdpath,
        'Author'         : author
    }
    with open('/content/info.txt','w') as f:
      f.write(title+'\n'+desc+'\n')

    return document

def manifest(start,end):
  path = '/content/contents/Nvideo.mp4'
  with open('/content/manifest.csv','w') as f:
    f.write('start_time,length,rename_to\n')
    f.write(f'{start},{end},{path}')
  print('WRITING TO MANIFEST FILE')

def split(meta,split_time,start,end):
  #SPLIT VIDEO
  path  = meta['Video_Path'] 

  splitter = '/content/video-splitter/ffmpeg-split.py'
  if os.path.isfile(splitter) :
    print('File is available. Moving along.')
  else:
    !git clone https://github.com/c0decracker/video-splitter.git
  if end == 0:
    subprocess.run(['python', splitter, '-s',split_time ,'-f', path ])
  else:
    manifest(start,end)
    subprocess.run(['python', splitter ,'-f', path, '-e','-copyinkf','-m','/content/manifest.csv' ])
    os. remove(path)

    path = '/content/contents/Nvideo.mp4'

    subprocess.run(['python', splitter, '-s', split_time  ,'-f', path])

  os. remove(path) 


def changeVideo(meta):
  lists = os.listdir('/content/contents/')
  author = meta['Author']
  x= 0
  paths = []
  for i in lists:
    paths.append(os.path.join('/content/contents',i))
    print(paths[x])
    x+=1
    regex = '(-\d-\d*)'
  info = '/content/info.txt'
  with open(info,'r') as f:
    p = f.readline()
  Name = p.replace('(Official Video)\n','').strip()
  path = paths[0]

  total = len(paths)
  num = 1
  for path in paths:
    part = re.findall('-(\w+)-',path)[0]
    try:
      annoy = '/content/contents/.ipynb_checkpoints'
      if os.path.isdir(annoy):
        os.removedirs(annoy)
      print( '---------------------------------------------')
      print(f'Video Part : {num} / {total}')
      print( '---------------------------------------------')
      videoE = VideoFileClip(path, audio=True)
      w,h = moviesize = videoE.size

      # txt = TextClip(, font='Amiri-regular',
      #                 color='yellow',fontsize=40);
      # txt = txt.set_position(lambda t:(10,50)).set_duration(videoE.duration);
      # video = CompositeVideoClip([videoE,txt.set_start(0.1).crossfadein(2)]);
      # # txt = TextClip(f'Part : {part} - {total}', font='Amiri-regular',
      # #                 color='yellow',fontsize=20);
      # # txt = txt.set_position(lambda t:(20,80)).set_duration(video.duration);
      # # video = CompositeVideoClip([video,txt.set_start(0.1).crossfadein(2)]);

      #texts after beginning screen
      txt01 = TextClip(f' {Name}', font='Amiri-regular',color='yellow',fontsize=30);
      txt01 = txt01.set_position(lambda t:(20,40)).set_duration(videoE.duration);
      txt02 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=30);
      txt02 = txt02.set_position(lambda t:(20,80)).set_duration(videoE.duration);
      #title
      title = TextClip(f'{Name}', font='Amiri-regular',color='yellow',fontsize=40,bg_color = 'black')
      title = title.set_position(('center','center')).set_duration(2)
      title01 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=40,bg_color='black');
      title01 = title01.set_position(('center',400)).set_duration(2);
      Author = TextClip(f'{author}', font='Amiri-regular',color='yellow',fontsize=20,bg_color='black');
      Author = Author.set_position(('center',450)).set_duration(2);
      video = CompositeVideoClip([videoE,title,title01,Author,txt01.set_start(3).crossfadein(1),txt02.set_start(3).crossfadein(1)]);
          
      video = video.rotate(angle = angle)

      name = Name.replace(' ','')
      video.write_videofile(f'/content/contents/{name}_{part}.mp4',progress_bar = False);

      # video.ipython_display(width=100,verbose=False,progress_bar = False)
      #clean up 
      os.remove(path)
    except:
      print(f'Something Wrong With Part : {part}')
    num+=1
def videoSavior():
  #Zip Folder and Download
  from google.colab import files
  !zip -r /content/contents/Videos.zip /content/contents/
  files.download("/content/contents/Videos.zip")

def cleanUP():
  dir = os.listdir('/content/contents/')
  print('Cleaning up Storage...')
  for i in dir:
    os.remove('/content/contents/'+i)
  print('Still Cleaning...')
  os.remove('/content/info.txt')
  os.remove('/content/manifest.csv')

def main(split_time,start,end):
    Link = input(' Please Enter YouTube Link : ')

    meta = download(Link)

    split(meta,str(split_time),start,end)

    changeVideo(meta)

    videoSavior()

    #cleanUP()

In [ ]:
# --------------------------------
#     --  Start Program   --

# Change these 2 variables only if you're splitting main video before splitting to 59s videos
start      = 13  # 0 default
end        = 380  # 0 default
split_time = 59 # time for videos. 59 secs for tiktoks

main(split_time,start,end) 

## Enter Desired Video to Upload to Status / Tiktok

In [ ]:
cleanUP()

In [ ]:
dir = os.listdir('/content/contents/')
for i in dir:
  os.remove('/content/contents/')
os.remove

In [ ]:

path = '/content/contents/Nvideo-6-of-6.mp4'
angle = 270
part = 10
name = 'THIS PROGRAM IS BEST FOR TIKTOK'
videoE = VideoFileClip(path, audio=True)
w,h = moviesize = videoE.size

#texts after beginning screen
txt01 = TextClip('I LOVE PYTHON ALOT', font='Amiri-regular',color='yellow',fontsize=40);
txt01 = txt01.set_position(lambda t:(10,20)).set_duration(videoE.duration);
txt02 = TextClip(f'Part : {part} - {10}', font='Amiri-regular',color='yellow',fontsize=40);
txt02 = txt02.set_position(lambda t:(10,70)).set_duration(videoE.duration);

title = TextClip(f'{Name}', font='Amiri-regular',color='yellow',fontsize=40,bg_color = 'black')
title = title.set_position(('center','center')).set_duration(2)
title01 = TextClip(f'Part : {part} - {10}', font='Amiri-regular',color='yellow',fontsize=40,bg_color='black');
title01 = title01.set_position((550,400)).set_duration(2);

#title
# start = (ImageClip('/content/pexels-roman-odintsov-6979824.jpg')
#           .set_duration(1)
#           .margin(right=w, top=h, opacity=0.20) # (optional) logo-border padding
#           .set_pos(("center","center")))

video = CompositeVideoClip([videoE,title,title01,txt01.set_start(3).crossfadein(1),txt02.set_start(3).crossfadein(1)]);
    
video = video.rotate(angle = angle)

video.write_videofile(f'/content/contents/{name}.mp4',progress_bar = False);
# video.ipython_display(width=100,verbose=False,progress_bar = False)

In [ ]:
print(h,w)

In [ ]:
def changeVideo1():
  lists = os.listdir('/content/contents/')
  author = 'meta'
  x= 0
  paths = []
  for i in lists:
    paths.append(os.path.join('/content/contents',i))
    print(paths[x])
    x+=1
    regex = '(-\d-\d*)'
  info = '/content/info.txt'
  with open(info,'r') as f:
    p = f.readline()
  Name = p.replace('(Official Video)\n','').strip()
  path = paths[0]

  total = len(paths)
  num = 1
  for path in paths:
    part = 9 #re.findall('-(\w+)-',path)[0]
    try:
      annoy = '/content/contents/.ipynb_checkpoints'
      if os.path.isdir(annoy):
        os.removedirs(annoy)
      print( '---------------------------------------------')
      print(f'Video Part : {num} / {total}')
      print( '---------------------------------------------')
      videoE = VideoFileClip(path, audio=True)
      w,h = moviesize = videoE.size

      # txt = TextClip(, font='Amiri-regular',
      #                 color='yellow',fontsize=40);
      # txt = txt.set_position(lambda t:(10,50)).set_duration(videoE.duration);
      # video = CompositeVideoClip([videoE,txt.set_start(0.1).crossfadein(2)]);
      # # txt = TextClip(f'Part : {part} - {total}', font='Amiri-regular',
      # #                 color='yellow',fontsize=20);
      # # txt = txt.set_position(lambda t:(20,80)).set_duration(video.duration);
      # # video = CompositeVideoClip([video,txt.set_start(0.1).crossfadein(2)]);

      #texts after beginning screen
      txt01 = TextClip(f' {Name}', font='Amiri-regular',color='yellow',fontsize=30);
      txt01 = txt01.set_position(lambda t:(20,40)).set_duration(videoE.duration);
      txt02 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=30);
      txt02 = txt02.set_position(lambda t:(20,80)).set_duration(videoE.duration);
      #title
      title = TextClip(f'{Name}', font='Amiri-regular',color='yellow',fontsize=40,bg_color = 'black')
      title = title.set_position(('center','center')).set_duration(2)
      title01 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=40,bg_color='black');
      title01 = title01.set_position(('center',400)).set_duration(2);
      Author = TextClip(f'{author}', font='Amiri-regular',color='yellow',fontsize=20,bg_color='black');
      Author = Author.set_position(('center',450)).set_duration(2);
      video = CompositeVideoClip([videoE,title,title01,Author,txt01.set_start(3).crossfadein(1),txt02.set_start(3).crossfadein(1)]);
          
      video = video.rotate(angle = angle)

      name = Name.replace(' ','')
      video.write_videofile(f'/content/contents/{name}_{part}.mp4',progress_bar = False);

      # video.ipython_display(width=100,verbose=False,progress_bar = False)
      #clean up 
      os.remove(path)
    except:
      print(f'Something Wrong With Part : {part}')
    num+=1

In [ ]:
path = '/content/contents/7SignsOfUnhealedRelationshipTrauma_6.mp4'
videoE = VideoFileClip(path, audio=True)
w,h = moviesize = videoE.size

# txt = TextClip(, font='Amiri-regular',
#                 color='yellow',fontsize=40);
# txt = txt.set_position(lambda t:(10,50)).set_duration(videoE.duration);
# video = CompositeVideoClip([videoE,txt.set_start(0.1).crossfadein(2)]);
# # txt = TextClip(f'Part : {part} - {total}', font='Amiri-regular',
# #                 color='yellow',fontsize=20);
# # txt = txt.set_position(lambda t:(20,80)).set_duration(video.duration);
# # video = CompositeVideoClip([video,txt.set_start(0.1).crossfadein(2)]);

#texts after beginning screen
txt01 = TextClip(f' {Name}', font='Amiri-regular',color='yellow',fontsize=30);
txt01 = txt01.set_position(lambda t:(20,40)).set_duration(videoE.duration);
txt02 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=30);
txt02 = txt02.set_position(lambda t:(20,80)).set_duration(videoE.duration);
#title
title = TextClip(f'{Name}', font='Amiri-regular',color='yellow',fontsize=40,bg_color = 'black')
title = title.set_position(('center','center')).set_duration(2)
title01 = TextClip(f'Part : {part} - {total}', font='Amiri-regular',color='yellow',fontsize=40,bg_color='black');
title01 = title01.set_position(('center',400)).set_duration(2);
Author = TextClip(f'{author}', font='Amiri-regular',color='yellow',fontsize=20,bg_color='black');
Author = Author.set_position(('center',450)).set_duration(2);
video = CompositeVideoClip([videoE,title,title01,Author,txt01.set_start(3).crossfadein(1),txt02.set_start(3).crossfadein(1)]);
  
video = video.rotate(angle = angle)

name = Name.replace(' ','')
video.write_videofile(f'/content/contents/{name}_{part}.mp4',progress_bar = False);
